In [114]:
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pickle
with open('/home/ema/dev/shocks/data/processed/AVAXBTC_5m.pkl', 'rb') as f:
    d = pickle.load(f)

d["features"]
data = [el for el in d["features"] if el["direction"] != 0]
non_shocks = [el for el in d["features"] if el["direction"] == 0]
data.extend(non_shocks[:len(data)])

binary = True
if binary:
    for el in data:
         direction = el['direction']
         el['direction'] = 1 if direction == -1 else direction

df = pd.DataFrame.from_dict(data).dropna()
cols = [col for col in df.columns if col != 'direction']
labels = df['direction'].tolist()
X_train, X_test, y_train, y_test = train_test_split(df[cols], labels, test_size=0.3, random_state=42, stratify=labels)

In [115]:
class_weights = {
   #-1: (len([el for el in labels if el == -1]) / len(labels)),
    0: (len([el for el in labels if el == 0]) / len(labels)),
    1: (len([el for el in labels if el == 1]) / len(labels)),
}

model = RandomForestClassifier(class_weight=class_weights)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.69      0.66      0.68       106
           1       0.67      0.70      0.69       105

    accuracy                           0.68       211
   macro avg       0.68      0.68      0.68       211
weighted avg       0.68      0.68      0.68       211



In [113]:
from sklearn.dummy import DummyClassifier

model = DummyClassifier(strategy="uniform")
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.52      0.57      0.54       106
           1       0.52      0.48      0.50       105

    accuracy                           0.52       211
   macro avg       0.52      0.52      0.52       211
weighted avg       0.52      0.52      0.52       211



In [33]:
import pystable

data = d["data"][["close"]].iloc[: 500]
np_data = np.array(data["close"].tolist())


init_fit = {"alpha": 2, "beta": 0, "sigma": 1, "mu": 0, "parameterization": 1}
dist = pystable.create(
    init_fit["alpha"],
    init_fit["beta"],
    init_fit["sigma"],
    init_fit["mu"],
    init_fit["parameterization"],
)

def fit(data, dist):
    for param in "alpha beta".split():
        data[param] = (
            data["close"]
            .rolling(250)
            .apply(lambda x: fit_levy(dist, x, param))
        )
    return data


def fit_levy(dist, log_returns, return_param):
    pystable.fit(dist, log_returns, len(log_returns))
    return getattr(dist.contents, return_param)


In [34]:
%%time
fitted = fit(data, dist)

CPU times: user 23.6 s, sys: 384 ms, total: 23.9 s
Wall time: 15.4 s


In [19]:
sliding_window_view(np.array(data), window_shape=10)


array([[0.0010223, 0.0010191, 0.0010214, ..., 0.000967 , 0.0009598,
        0.0009489],
       [0.0010191, 0.0010214, 0.0009966, ..., 0.0009598, 0.0009489,
        0.0009452],
       [0.0010214, 0.0009966, 0.0009912, ..., 0.0009489, 0.0009452,
        0.000959 ],
       ...,
       [0.0021063, 0.002076 , 0.0021465, ..., 0.002073 , 0.0020889,
        0.0020967],
       [0.002076 , 0.0021465, 0.0020999, ..., 0.0020889, 0.0020967,
        0.0021095],
       [0.0021465, 0.0020999, 0.0020906, ..., 0.0020967, 0.0021095,
        0.0021405]])